# RRT (Rapidly-exploring Random Tree) and Bidirectional-RRT # 

In [ ]:
import random
import math
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon
import imageio
import io
import numpy as np
from shapely.geometry import Polygon as ShapelyPolygon
from shapely.geometry import Point
from shapely.geometry import Point as ShapelyPoint
from shapely.geometry import LineString

In [ ]:
class Node:
    def __init__(self, x, y):
        self.x = x
        self.y = y 
        self.parent = None

The Node class represents the nodes in the tree. Each node has an attribute to store the position (x, y) in the space and an attribute for the parent node.

In [ ]:
def draw_background():
    plt.figure(figsize=(10, 8))
    plt.xlim(0, 100)
    plt.ylim(0, 100)
    plt.xlabel('X')
    plt.ylabel('Y')
    plt.title('Robot Motion Planning: RRT')
    plt.grid(False)

def generate_random_polygon_obstacle(num_vertices_range, x_range, y_range):
    num_vertices = np.random.randint(*num_vertices_range) # unpacking
    vertices = np.random.rand(num_vertices, 2) * [x_range, y_range]
    obstacle = Polygon(vertices, closed=True, edgecolor='black', facecolor='gray')
    return obstacle

def draw_polygon_obstacles(obstacles):
    for obstacle in obstacles:
        polygon = Polygon(obstacle.get_xy(), closed=True, edgecolor='black', facecolor='gray')
        plt.gca().add_patch(polygon)

__*draw_background()*__ : Creates a new plot with specific dimensions (10 inches by 8 inches), sets the X and Y axis limits to range from 0 to 100, labels the X and Y axes. 

__*generate_random_polygon_obstacle(num_vertices_range, x_range, y_range)*__ : Generates a random polygon obstacle with a random number of vertices within a specified range (num_vertices_range) and random coordinates within specified ranges (x_range and y_range). Each vertex is generated with random coordinates and used to create a polygon obstacle with specified characteristics (closed shape, black border, gray face). 

__*draw_polygon_obstacles(obstacles)*__ : Draws a collection of polygon obstacles onto the current plot. It iterates over each obstacle in the obstacles list and adds each one as a patch to the current plot.

In [ ]:
def generate_random_point(min_xy, max_xy):
    x = random.uniform(min_xy, max_xy)
    y = random.uniform(min_xy, max_xy)
    return Point(x, y)

def distance(node_x, node_y, point_x, point_y):
    return math.sqrt((node_x - point_x)**2 + (node_y - point_y)**2)

def find_nearest_node(tree, random_point_x, random_point_y):
    min_dist = math.inf
    nearest_node = None
    for node in tree:
        dist = distance(node.x, node.y, random_point_x, random_point_y)
        if dist < min_dist:
            min_dist = dist
            nearest_node = node
    return nearest_node

__*def generate_random_point(min_x, max_x, min_y, max_y)*__ : Generates a random point in the space. 

__*def distance(x, y)*__ : Calculates the distance between two positions. 

__*def find_nearest_node(tree, random_point)*__ : Given a point find the nearest node in the tree. 

In [ ]:
def is_collision_free(point, obstacles):
    point_shapely = ShapelyPoint(point.x, point.y)
    for obstacle in obstacles:
        obstacle_shapely = ShapelyPolygon(obstacle.get_xy())
        if obstacle_shapely.contains(point_shapely):
            return False
    return True

def is_path_collision_free(start_point, end_point, obstacles):
    path_line = LineString([(start_point.x, start_point.y), (end_point.x, end_point.y)])
    for obstacle in obstacles:
        obstacle_polygon = ShapelyPolygon(obstacle.get_xy())
        if path_line.intersects(obstacle_polygon):
            return False
    return True

__*def is_collision_free(point, obstacles)*__ : This function checks whether given a point does not overlap with any of the obstacles in the environment.

__*def is_path_collision_free(start_point, end_point, obstacles)*__ : This function checks whether a straight line (or segment) between two points does not cross any of the obstacles in the environment.

In both functions, the Shapely module is used to represent points and polygons and to perform geometric operations such as containment and intersection checking. Obstacles are represented as polygons, while the path is represented as a straight line between two points.

In [ ]:
def choose_new_point(random_point, nearest_node, obstacles):
    # Calculate the length of the segment.
    dist_x = abs (random_point.x - nearest_node.x)
    dist_y = abs (random_point.y -nearest_node.y)
    length_segment = max (dist_x, dist_y)
    dist_max = []
    # Generates points along the segment.
    points = []
    for i in range(int(length_segment) + 1):
        x = round(random_point.x + (i / length_segment) * (nearest_node.x - random_point.x))
        y = round(random_point.y + (i / length_segment) * (nearest_node.y - random_point.y))
        points.append((x, y)) 
    for p in points:
        dist = distance(nearest_node.x, nearest_node.y, p[0] , p[1])
        dist_max.append((dist, p))
    dist_max = sorted(dist_max, key=lambda x: x[0], reverse=True)
    for i in dist_max:
            if is_path_collision_free(Point(i[1][0], i[1][1]), nearest_node, obstacles):
                return i[1]
        
def add_node_to_tree(new_node, nearest_node):
    new_node.parent = nearest_node

def extend_tree(tree, qnew):
    tree.append(qnew)      

__*def choose_new_point(random_point, nearest_node, obstacles)*__ : This function chooses a new point along the segment between the random point and the nearest node. It calculates the length of the segment using the difference between the x and y coordinates of the random point and the nearest node, then determines the maximum length between these two differences.It generates a series of points along the segment using the linear interpolation formula.
For each point generated along the segment, calculate the distance between the point and the nearest node.

__*def add_node_to_tree(new_node, nearest_node)*__ : This function adds the newly created node (arising from the chosen point) as a child of the nearest node.

__*def extend_tree(tree, qnew)*__ : This function extends the tree by adding the new node to it.

In [ ]:
def rrt(num_iterations, obstacles, start_point, goal_point):
    tree = []
    tree_list = []
    tree.append(Node(start_point.x, start_point.y))
    tree_list.append(tree.copy())

    for _ in range(num_iterations):
        random_point = generate_random_point(0, 100)
        nearest_node = find_nearest_node(tree, random_point.x, random_point.y)
        qs = choose_new_point(random_point, nearest_node, obstacles)
        if qs is None:
            continue
        new_node = Node(qs[0], qs[1])
        add_node_to_tree(new_node, nearest_node)
        extend_tree(tree, new_node)

        # Checking whether the newly added node is close to the target point
        if distance(new_node.x, new_node.y, goal_point.x, goal_point.y) < 2:
            # If the newly added node is close to the end point, add the end point as a child of the newly added node
            goal_node = Node(goal_point.x, goal_point.y)
            add_node_to_tree(goal_node, new_node)
            tree_list.append(tree.copy())  
            return tree_list
           
        
        tree_list.append(tree.copy())  
    return tree_list

__*def rrt(num_iterations, obstacles, start_point, goal_point)*__ : This function implements the RRT (Rapidly-exploring Random Tree) algorithm for finding paths in a configuration space.
An empty tree and a list is initialized that will contain the states of the tree at each iteration.
The starting point is added as an initial node to the tree.
A loop is executed for a specified number of iterations (num_iterations).
At each iteration, a random point in the environment is generated.
The nearest node in the tree to the generated random point is found.
A new point is chosen along the segment between the random point and the nearest node, avoiding collisions with obstacles.
If the new point does not cause collisions, it is added to the tree as a new node.
The new node is extended in the tree.
It is checked whether the new node is sufficiently close to the target point (goal_point). If it is, it is added as a child of the new node and the algorithm terminates by returning the tree.
If the maximum number of iterations is reached without the goal point being reached, the algorithm still returns the current tree.


In [ ]:
def get_path_to_root(node):
    path = []
    while node is not None:
        path.append(node)
        node = node.parent
    return list(reversed(path))  # Invert the list to get the path from the root to the current node

def check_intersection(tree_start, tree_goal, obstacles):
    for node_start in tree_start:
        for node_goal in tree_goal:
            if is_path_collision_free(node_start, node_goal, obstacles):
                return (node_start, node_goal)
    return None

def rrt_bidirectional(num_iterations, obstacles, start_point, goal_point, threshold_distance=2.0):
    tree_start = [Node(start_point.x, start_point.y)]
    tree_goal = [Node(goal_point.x, goal_point.y)]
    final_path = None
    rrt_trees = []

    for iteration in range(1, num_iterations + 1):
        random_point_start = generate_random_point(0, 100)
        nearest_node_start = find_nearest_node(tree_start, random_point_start.x, random_point_start.y)
        qs_start = choose_new_point(random_point_start, nearest_node_start, obstacles)
        if qs_start is not None:
            new_node_start = Node(qs_start[0], qs_start[1])
            add_node_to_tree(new_node_start, nearest_node_start)
            extend_tree(tree_start, new_node_start)

        random_point_goal = generate_random_point(0, 100)
        nearest_node_goal = find_nearest_node(tree_goal, random_point_goal.x, random_point_goal.y)
        qs_goal = choose_new_point(random_point_goal, nearest_node_goal, obstacles)
        if qs_goal is not None:
            new_node_goal = Node(qs_goal[0], qs_goal[1])
            add_node_to_tree(new_node_goal, nearest_node_goal)
            extend_tree(tree_goal, new_node_goal)

        # Check for intersection between the latest nodes added to the trees
        latest_nodes_start = tree_start[-5:]  # Consider the 5 latest nodes (or all nodes if less than 5)
        latest_nodes_goal = tree_goal[-5:]  # Consider the 5 latest nodes (or all nodes if less than 5)
        for node_start in latest_nodes_start:
            for node_goal in latest_nodes_goal:
                if distance(node_start.x, node_start.y, node_goal.x, node_goal.y) < threshold_distance:
                    path_start = get_path_to_root(node_start)
                    path_goal = get_path_to_root(node_goal)
                    path_goal.reverse()
                    final_path = path_start + path_goal
                    return final_path, rrt_trees

        if iteration % 10 == 0:
            rrt_trees.append((tree_start.copy(), tree_goal.copy()))

    return final_path, rrt_trees



__*get_path_to_root(node)*__ : This function returns the path from the root of a tree to the specified node. It is used to obtain the path from the root to the current node.

__*check_intersection(tree_start, tree_goal, obstacles)*__ : This function checks whether there is an intersection between the two trees (source tree and target tree). It scans pairs of nodes from the two trees and checks whether the path between them is free of collisions with obstacles.

__*rrt_bidirectional(num_iterations, obstacles, start_point, goal_point, threshold_distance=2.0)*__ : This is the main function that implements the bidirectional RRT algorithm. It iterates for a specified number of times (num_iterations), randomly generating points in the working environment and trying to extend the search trees toward them. When the search trees meet within a certain distance (threshold_distance), the final path is constructed by combining the paths from the root to the intersecting nodes in the two trees.

These functions form a basic implementation of the bidirectional RRT algorithm for path planning.

**Execution of the RRT algorithm**:

In [ ]:
custom_light_green = (0.4, 0.6, 0.4)  
custom_light_blue = (0.4, 0.4, 0.6) 

x_range = (0, 100)
y_range = (0, 100)
# will generate a random number between 3 and 5, inclusive.
num_obstacles = np.random.randint(3, 6) 
obstacles = []
image_files = []

iteration = 0
while iteration < num_obstacles:
    x_range = np.random.randint(30, 100)
    y_range = np.random.randint(60, 100)
    obstacle = generate_random_polygon_obstacle((3, 5), x_range, y_range)

# checking for possible collisions
    overlapping = False
    obstacle_shapely = ShapelyPolygon(obstacle.get_xy())
    for prev_obstacle in obstacles: 
        prev_obstacle_shapely = ShapelyPolygon(prev_obstacle.get_xy())
        if obstacle_shapely.intersects(prev_obstacle_shapely):
            overlapping = True
            break
    if not overlapping:
        obstacles.append(obstacle)
        iteration += 1

draw_background()
draw_polygon_obstacles(obstacles)


s = False
g = False

while s == False:
    start_point = generate_random_point(0, 35) 
    if is_collision_free(start_point, obstacles):
        plt.plot(start_point.x, start_point.y, marker='o', markersize=10, color= "green", label='Start')
        s = True
        break  

while g == False:
    goal_point = generate_random_point(60, 100) 
    if is_collision_free(goal_point, obstacles):
        plt.plot(goal_point.x, goal_point.y, marker='o', markersize=10, color= "blue", label='Goal')
        g = True
        break  

# Eexecution of the RRT algorithm
rrt_trees = rrt(1000, obstacles, start_point, goal_point)

# Find the final path
final_path = []
current_node = rrt_trees[-1][-1]  # Last node of the last RRT tree
while current_node:
    final_path.append((current_node.x, current_node.y))
    current_node = current_node.parent



gif_filename = 'rrt_animation.gif'
with imageio.get_writer(gif_filename, mode='I') as writer:
    for i, tree in enumerate(rrt_trees):
        print(f"Iteration {i+1}/{len(rrt_trees)}")  # Print the current iteration number
        
        plt.figure(figsize=(10, 8))
        draw_background()
        draw_polygon_obstacles(obstacles)
        for node in tree:
            if node.parent:
                plt.plot([node.x, node.parent.x], [node.y, node.parent.y], color=custom_light_green, linewidth=3)
        plt.plot(start_point.x, start_point.y, marker='o', markersize=10, color="green", label='Start')
        plt.plot(goal_point.x, goal_point.y, marker='o', markersize=10, color="blue", label='Goal')
        plt.legend()
        plt.title(f'RRT Iteration {i+1}')
        
        buf = io.BytesIO()
        plt.savefig(buf, format='png')
        buf.seek(0)
        image = imageio.imread(buf)
        writer.append_data(image)
        
        plt.close()  

    if final_path:
        plt.figure(figsize=(10, 8))
        draw_background()
        draw_polygon_obstacles(obstacles)
        for node in tree:
            if node.parent:
                plt.plot([node.x, node.parent.x], [node.y, node.parent.y], color=custom_light_green, linewidth=3)
        plt.plot(start_point.x, start_point.y, marker='o', markersize=10, color="green", label='Start')
        plt.plot(goal_point.x, goal_point.y, marker='o', markersize=10, color="blue", label='Goal')
        path_x, path_y = zip(*final_path)
        plt.plot(path_x, path_y, color='red', linewidth=4, alpha=0.7, label='Final Path')
        plt.legend()
        plt.title(f'Final Path: Iteration {i+1}')
        
        buf = io.BytesIO()
        plt.savefig(buf, format='png')
        buf.seek(0)
        image = imageio.imread(buf)
        writer.append_data(image)
        
        plt.close()  

print(f"GIF created: {gif_filename}")


__*Obstacle generation:*__

Initially, the x- and y-axis range (x_range and y_range) are defined and a random number of obstacles between 3 and 5 (inclusive) are generated.
A loop is run until all desired obstacles are generated, ensuring that they do not overlap with previous obstacles.
The obstacles are represented as randomly generated polygons using the generate_random_polygon_obstacle function.

__*Generation of start_point and goal_point:*__

A start point (start_point) and destination point (goal_point) are randomly generated, ensuring that both do not overlap with obstacles.

__*Execution of the RRT algorithm:*__

The RRT function is called, which takes as input the desired number of iterations, obstacles, starting point and destination point.
The RRT algorithm is executed for a specified number of iterations, adding nodes to the RRT tree in order to explore the search space. An animated GIF file is created to visualize the evolution of the RRT algorithm during iterations.

In [ ]:
# Execution of the RRT bidirectional algorithm
final_path, rrt_trees = rrt_bidirectional(1000, obstacles, start_point, goal_point)

# Create gif
gif_filename = 'rrt_bidirectional_animation.gif'

# Create gif frames
with imageio.get_writer(gif_filename, mode='I') as writer:
    # Draw initial background without any paths
    plt.figure(figsize=(10, 8))
    draw_background()
    draw_polygon_obstacles(obstacles)
    plt.plot(start_point.x, start_point.y, marker='o', markersize=10, color='green', label='Start')
    plt.plot(goal_point.x, goal_point.y, marker='o', markersize=10, color='blue', label='Goal')
    plt.legend()
    plt.title('Initial State')
    
    # Save initial frame
    buf = io.BytesIO()
    plt.savefig(buf, format='png')
    buf.seek(0)
    image = imageio.imread(buf)
    writer.append_data(image)

    # Create gif frames for each iteration
    for i, (tree_start, tree_goal) in enumerate(rrt_trees):
        print(f"Iteration {i+1}/{len(rrt_trees)}")  # Print the current iteration number
        plt.figure(figsize=(10, 8))
        draw_background()
        draw_polygon_obstacles(obstacles)
        for node_start in tree_start:
            if node_start.parent:
                plt.plot([node_start.x, node_start.parent.x], [node_start.y, node_start.parent.y], color=custom_light_green, linewidth=3)
        for node_goal in tree_goal:
            if node_goal.parent:
                plt.plot([node_goal.x, node_goal.parent.x], [node_goal.y, node_goal.parent.y],  color=custom_light_blue, linewidth=3)
        plt.plot(start_point.x, start_point.y, marker='o', markersize=10, color='green', label='Start')
        plt.plot(goal_point.x, goal_point.y, marker='o', markersize=10, color='blue', label='Goal')
        
        # Trace the final route in red if it exists
        if i == len(rrt_trees) - 1 and final_path:
            for node_start, node_goal in zip(final_path[:-1], final_path[1:]):
                plt.plot([node_start.x, node_goal.x], [node_start.y, node_goal.y], color='red', linewidth=4, alpha=1.0)
        
        plt.legend()
        plt.title(f'RRT Bidirectional Iteration {i+1}')
        
        # Get the bytes of the image data
        buf = io.BytesIO()
        plt.savefig(buf, format='png')
        buf.seek(0)
        image = imageio.imread(buf)
        writer.append_data(image)
        
        plt.close()  

print(f"GIF created: {gif_filename}")

__*Execution of the RRT_bidirectional algorithm:*__

This code block executes the bidirectional RRT algorithm to find an optimal path between a starting point and an ending point, taking into account obstacles in the environment. The algorithm generates a series of frames showing the evolution of the RRT trees created during execution, along with the final path found. Initially, the algorithm performs *num_iterations* iterations of the bidirectional RRT, adding nodes to the RRT trees starting from the starting point and ending point. At each iteration, it checks whether the last nodes added to the trees are sufficiently close to each other. If yes, a final path is constructed by combining the paths from the root trees to the neighboring nodes.
Next, frames are created for the GIF showing the evolution of the RRT trees over time. These frames are generated using imageio to create the animated GIF. Each frame shows the current state of the RRT trees, with the nodes represented as points in the graph and the paths between nodes represented by lines.